In [3]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import geopandas as gpd
from shapely.geometry import Point, mapping
%matplotlib inline

import warnings
warnings.simplefilter(action="ignore")

pd.set_option('precision', 5)
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

import cityImage as ci
import ABManalysis as af

### Preparing and cleaning the routes

In [4]:
city_name = 'Muenster'
epsg = 25832
crs = 'EPSG:'+str(epsg)

In [297]:
rs = gpd.read_file('input/routesAll.shp')
dp = gpd.read_file('input/decisionPoints.shp')

In [ ]:
routes = rs.copy()
routes = routes[0:0]

for nr in range(3):
    tmp = rs[rs.routeID == nr].copy()
    tmp_dp = dp[dp.routeID == nr].copy() 
    tmp_dp = tmp_dp[(tmp_dp.nodeID != tmp.O.iloc[0] ) & (tmp_dp.nodeID != tmp.D.iloc[0] )]
    gdf = split_at_dp(tmp, tmp_dp)
    gdf.reset_index(inplace = True, drop = True)
   
    processed = []
    for r in gdf.itertuples():
        if r.Index in processed: continue
        geo = gdf.loc[r.Index].geometry
        routeChoice = gdf.loc[r.Index].routeChoic
        rc = []
        
        for rr in gdf.itertuples():
            if r == rr: continue
            elif geo == gdf.loc[rr[0]].geometry:
                other_rc = gdf.loc[rr[0]].routeChoic
                rc.append(other_rc)
                processed.append(rr.Index)
        
        if len(rc)> 0:
            rc.append(routeChoice)
            gdf.at[r.Index, 'routeChoic'] = rc
        else:
            gdf.at[r.Index, 'routeChoic'] = [routeChoice]
        
        routes = routes.append(gdf.loc[r.Index])
    routes.reset_index(inplace = True, drop = True)

In [300]:
routes['length'] = routes.geometry.length
routes['routeChoic'] = [','.join(map(str, l)) for l in routes['routeChoic']]
routes.to_file("Outputs/split.shp", driver='ESRI Shapefile')

In [ ]:
def split_at_dp(gdf, dp):
    old_gdf = gdf.copy()
    
    for row in tmp.itertuples():
        line_geometry = gdf.loc[row.Index].geometry
        union = dp.unary_union
        intersections = line_geometry.intersection(union)
        if intersections.geom_type == 'Point': 
            intersections = [intersections]

        points = [i for i in intersections if i.geom_type == 'Point']
        new_collection = []

        for p in points:
            if (p.coords[0] == line_geometry.coords[0]) | (p.coords[0] == line_geometry.coords[-1]): 
                pass # disregarding the ones which lie on the line's u-v nodes
            else: new_collection.append(p) # only checking the others
        if len(new_collection) == 0: 
            continue    

        geometry_collection = MultiPoint([point.coords[0] for point in new_collection])  
        # including the intersecting geometries in the coordinates sequence of the line and split
        new_line_geometries = ci.split_line_at_MultiPoint(line_geometry, geometry_collection) 
        len(new_line_geometries)

        for n, line in enumerate(new_line_geometries): # assigning the resulting geometries
            if n == 0: 
                index = row.Index
            else: index = max(gdf.index)+1
            # copy attributes
            gdf.loc[index] = gdf.loc[row.Index]  
            # and assign geometry an new edgeID 
            gdf.at[index, 'geometry'] = line

    return gdf  